In [1]:
from langchain_community.document_loaders import WebBaseLoader, AsyncHtmlLoader

loader = WebBaseLoader("https://www.espn.com/")
loader.requests_kwargs = {"verify": False}

docs = loader.load()

/home/codespace/.python/current/lib/python3.10/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.espn.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [15]:
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader


from bs4 import BeautifulSoup as Soup

url = "https://huggingface.co/"
loader = RecursiveUrlLoader(
    url=url, max_depth=2, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()



/home/codespace/.python/current/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


In [11]:
from pprint import pprint

In [16]:
print(docs[0].page_content)






















 
Hugging Face – The AI community building the future.






Hugging Face







					Models

					Datasets

					Spaces

					Posts

					Docs




			Solutions
		

Pricing
			






Log In
				
Sign Up
					


NEW
Create Assistants in HuggingChat
				

The AI community building the future.
			
The platform where the machine learning community collaborates on models, datasets, and applications.
			






			Trending onthis week
			


					Models
				

google/gemma-7b


Updated
				about 12 hours ago
• 

				142k
			• 

				1.57k
			
ByteDance/SDXL-Lightning


Updated
				2 days ago
• 

				2.2k
			• 

				804
			
google/gemma-7b-it


Updated
				6 days ago
• 

				53.4k
			• 

				759
			
google/gemma-2b


Updated
				7 days ago
• 

				32.4k
			• 

				436
			
google/gemma-2b-it


Updated
				2 days ago
• 

				36.2k
			• 

				326
			
Browse 400k+ models

					Spaces
				




			Running
			onzero



1.04k
👁
Stable Cascade





			Running
			onzero



282
⚡
SDXL L

In [34]:
docs[0].page_content

'<!doctype html><html lang="en"><head><meta charset="utf-8"/><link rel="icon" href="/favicon.ico"/><meta name="viewport" content="width=device-width,initial-scale=1,user-scalable=no"/><meta name="theme-color" content="#000000"/><meta name="description" content="MDC Software Website"/><link rel="apple-touch-icon" href="/logo192.png"/><link rel="manifest" href="/manifest.json"/><title>MDC Software</title><link href="/static/css/2.98c681a9.chunk.css" rel="stylesheet"><link href="/static/css/main.630a7d1a.chunk.css" rel="stylesheet"></head><body><noscript>You need to enable JavaScript to run this app.</noscript><div id="root"></div><script>!function(e){function t(t){for(var n,i,a=t[0],c=t[1],l=t[2],s=0,p=[];s<a.length;s++)i=a[s],Object.prototype.hasOwnProperty.call(o,i)&&o[i]&&p.push(o[i][0]),o[i]=0;for(n in c)Object.prototype.hasOwnProperty.call(c,n)&&(e[n]=c[n]);for(f&&f(t);p.length;)p.shift()();return u.push.apply(u,l||[]),r()}function r(){for(var e,t=0;t<u.length;t++){for(var r=u[t],n=

In [38]:
from bs4 import BeautifulSoup as Soup
import requests

url = "https://www.mdcsoftware.com.vn/"

page= requests.get(url)
# soup = Soup(page.content, "html.parser")

In [41]:
page.text

'<!doctype html><html lang="en"><head><meta charset="utf-8"/><link rel="icon" href="/favicon.ico"/><meta name="viewport" content="width=device-width,initial-scale=1,user-scalable=no"/><meta name="theme-color" content="#000000"/><meta name="description" content="MDC Software Website"/><link rel="apple-touch-icon" href="/logo192.png"/><link rel="manifest" href="/manifest.json"/><title>MDC Software</title><link href="/static/css/2.98c681a9.chunk.css" rel="stylesheet"><link href="/static/css/main.630a7d1a.chunk.css" rel="stylesheet"></head><body><noscript>You need to enable JavaScript to run this app.</noscript><div id="root"></div><script>!function(e){function t(t){for(var n,i,a=t[0],c=t[1],l=t[2],s=0,p=[];s<a.length;s++)i=a[s],Object.prototype.hasOwnProperty.call(o,i)&&o[i]&&p.push(o[i][0]),o[i]=0;for(n in c)Object.prototype.hasOwnProperty.call(c,n)&&(e[n]=c[n]);for(f&&f(t);p.length;)p.shift()();return u.push.apply(u,l||[]),r()}function r(){for(var e,t=0;t<u.length;t++){for(var r=u[t],n=

In [44]:
from playwright import sync_playwright

soup = None
def scrape_website(url):
    with sync_playwright() as p:
        global soup
        browser = p.chromium.launch()
        context = browser.new_context()
        page = context.new_page()

        # Navigate to the website
        page.goto(url)

        # Wait for the page to load (you may need to adjust the waiting time)
        page.wait_for_load_state()

        # Get the page content after JavaScript execution
        page_content = page.content()

        # Now you can use BeautifulSoup to parse the page content
        soup = Soup(page_content, "html.parser")

        # Perform scraping using BeautifulSoup on the dynamically rendered content
        # ...

        # Close the browser when done
        context.close()
        browser.close()



# Replace 'https://example.com' with the URL of the website you want to scrape
scrape_website(url)

ImportError: cannot import name 'sync_playwright' from 'playwright' (/home/codespace/.python/current/lib/python3.10/site-packages/playwright/__init__.py)

In [46]:
import asyncio
from pyppeteer import launch
from bs4 import BeautifulSoup
import nest_asyncio
page = None
async def main(url):
    global page
    browser = await launch()
    page = await browser.newPage()
    # url = 'https://example.com'
    # url = "https://www.mdcsoftware.com.vn/"
    await page.goto(url)
    
    # You may need to wait for specific elements to load
    # await page.waitForSelector('your-selector')

    # Get the page content after JavaScript execution
    page_content = await page.content()

    # Close the browser
    await browser.close()

    # Now you can use BeautifulSoup to parse the page content
    soup = BeautifulSoup(page_content, 'html.parser')

    # Perform scraping using BeautifulSoup on the dynamically rendered content
    # ...return soup
    # print(soup.prettify())
    return soup

nest_asyncio.apply()
url = "https://www.mdcsoftware.com.vn/"
soup = asyncio.get_event_loop().run_until_complete(main(url))



In [52]:
# split by uppercase characters
import re
# re.findall('[A-Z][^A-Z]*', soup.text)
soup.text

'MDC SoftwareYou need to enable JavaScript to run this app.HomeProductServicesRecruitmentContactEnglishOur specialized teamsWe combine strategy, technology, data science and design to solve the most complex business challenges.Scroll to see moreWHO ARE WE?Our specialized teams lead the product design and development process for high growth companies.At MDC, we always appreciate the smallest ideas, you will be completely assured with a clear, fast workflow. \nWhile our project consultancy is completely free of charge, we offer extensive QA process of highest quality to our clients with reasonable price. Our experienced team of engineers are always enthusiastic in taking up new challenges and offering our clients with the best service. In addition, each project is carefully planned to stay consistent with customers’ plans. Plus, our 24/7 customer support is also one of the merits for customers. Web ApplicationMDC SOFTWARE committed to build highest quality applicationArtificial Intellige

In [55]:
pages = []
links = soup.find_all("a")
for link in links:
    if re.match(r'^/.', link.get("href")):
        pages.append(asyncio.get_event_loop().run_until_complete(main(url[:-1] + link.get("href"))))
        # print(soup.text)
        

In [68]:
pages[3].text

'MDC SoftwareYou need to enable JavaScript to run this app.HomeProductServicesRecruitmentContactEnglishIf your actions inspire others to dream more, learn more, do more and become more, you are a leader.MDC MEDIASEND AN APPOINTMENTPlease complete the details below and then click on Submit and we’ll be in contactSendDo you intend to find companies writing mobile applications?MDC Media is working in Hanoi City specializing in mobile development and web design for individuals, companies and startups. We consist of members specializing in production and with the motto "One app at a time".Request Application Quote MDC VIETNAM MEDIA AND TECHNOLOGY COMPANY LIMITED15th Floor, Viet A Building, No. 9 Duy Tan, Cau Giay, Hanoi0868.733.900hello@mdcsoftware.com.vnCopyright © 2021 MDC MEDIA CO.,LTD. All Rights Reserved.'

In [69]:
overview = """Company Overview:
Name: MDC Media
Location: Hanoi City
Specialization: Mobile development, web design
Motto: "One app at a time"
"""

In [79]:
import os
import openai
import sys

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings.openai import OpenAIEmbeddings

# vectorstore = FAISS.fro
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

In [85]:
import langchain
langchain.debug = True

In [86]:
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-3.5-turbo-1106", temperature=0)

retrieval_chain = prompt | model | StrOutputParser()

# retrieval_chain.invoke("where did harrison work?")

# chain.invoke("What did the president say about technology?")

In [ ]:
retrieval_chain.invoke({"context": information , "question":"How to contact to MDC?"})